In [1]:
import Torch

In [2]:
input = asTensor ([1..1000] :: [Int])
shape input

[1000]

In [3]:
type DataLoader = [(Tensor, Tensor)]

In [7]:
import Torch.Functional as F
import Torch.Functional.Internal as FI

In [ ]:
chunksOf :: Int -> [a] -> [[a]]
chunksOf _ [] = []
chunksOf n xs = let (chunk, rest) = splitAt n xs in chunk : chunksOf n rest

createDataLoader
  :: Int     
  -> Int     
  -> Tensor  
  -> DataLoader
createDataLoader batchSize seqLen tokens = 
  let 
    tokenSize = Torch.size 0 tokens 
    
    createInputs = 
      [ FI.slice tokens 0 i (i + seqLen) 1
      | i <- [0 .. tokenSize - seqLen - 1] ]
    
    createTargets = 
      [ FI.slice tokens 0 (i + 1) (i + seqLen + 1) 1
      | i <- [0 .. tokenSize - seqLen - 1] ]
    
    dataset = zip createInputs createTargets
    
    batches = filter (\chunk -> length chunk == batchSize) (chunksOf batchSize dataset)
    
    processBatch batch =
      let inputs = map fst batch
          targets = map snd batch
          inputBatch = Torch.stack (Dim 0) inputs    
          targetBatch = Torch.stack (Dim 0) targets  
      in (inputBatch, targetBatch)
    
  in map processBatch batches
